# HW 1: Corporate Bond Pricing (due by 9.17 Tues)

We are going to compute the price of a corporate bond (subject to default) with Monte-Carlo simulation. Assume that 
* the default time of a company follows the exponential distribution with intensity $\lambda=$__`def_rate`__. 
* the riskfree interest rate is $r_f=$__`rf_rate`__ and the maturity of the bond is $T=$__`mat`__. 
* in the case of default, you can recover some portion ($R=$__`recovery_rate`__) of the face value.
* the coupon is 0%, i.e., it is a zero-coupon bond.
* the face value of the bond is 1.0
* use compound rate for discounting; the price of the default-free bond is $e^{-r_f T}$

The Problem 1 of the [2017 ASP Midterm Exam](../files/ASP2017_Midterm.pdf) will be helpful.

### Instruction to upload your HW
* Create a repository named __`PHBS_ASP_2019`__ (and clone it to your PC)
* Copy this file to __`PHBS_ASP_2019/HW1/HW1.ipynb`__  (Please use the same name for repository and ipynb file)
* Add solution code.
* Run your your code to make sure that there's no error.
* Upload (commit and sync) your file.

### 1. First, let's create a pricing function and check the std 

In [2]:
import numpy as np

In [3]:
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10

In [4]:
# First generate exponential random numbers
# Although you can generate directly using fault_time = np.random.exponential(scale=), let's use uniform random numbers.
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(U)
# You can check if the RNs are correct by comparing the means
(default_time.mean(), 1/def_rate)

(9.80339208521233, 10.0)

In [5]:
# Put your code here to price the corporate bond

def corp_bond(mat=10, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=1e4):
    U = np.random.uniform(size=n_sample)
    default_time = -(1/def_rate)*np.log(U)
    payment=((default_time-mat)>=0)*np.exp(-rf_rate*mat)+((default_time-mat)<0)*recovery*np.exp(-rf_rate*default_time)
    price=payment.mean()
    return price

# Call your function
corp_bond(mat, def_rate, rf_rate, recovery, n_sample)

# Find the mean and std by calling the function 100 times. 
#Number of Simulation
NumSim=100
#ResultArray
ResArr_0=np.zeros(shape=NumSim)
for i in range(0,NumSim):
    ResArr_0[i]=corp_bond(mat, def_rate, rf_rate, recovery, n_sample)
    i=i+1
(ResArr_0.mean(),ResArr_0.std())

(0.4399721807668823, 0.002167997353448178)

### 2. Now, let's improve the function by reducing the MC variations.
1. Use antithetic method: If `U` is uniform random variable, so is `1-U`
2. Also shift the RNs to match the mean, `1/def_rate`

In [6]:
# For example, antithetic method mean
n_sample = 10000
U = np.random.uniform(size=int(n_sample/2))
default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))

# Mean-matching means
default_time += 1/def_rate-default_time.mean()
(default_time.mean(), 1/def_rate)

(10.0, 10.0)

In [7]:
# No include the two new features: `antithetic` and `mean_match`

def corp_bond_cv(mat=10, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=1e4, antithetic=True, mean_match=True):
    
    if(antithetic):
        U = np.random.uniform(size=int(n_sample/2))
    else:
        U = np.random.uniform(size=n_sample) 
        
    default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
    
    if(mean_match):
        default_time += 1/def_rate-default_time.mean()
    
    payment=((default_time-mat)>=0)*np.exp(-rf_rate*mat)+((default_time-mat)<0)*recovery*np.exp(-rf_rate*default_time)
    price=payment.mean()   

    return price

# Find the mean and std by calling the function 100 times for (i) antithetic (ii) mean_match and (iii) both
ResArr_1=np.zeros(shape=NumSim)
ResArr_2=np.zeros(shape=NumSim)
ResArr_3=np.zeros(shape=NumSim)
for i in range(0,NumSim):
    ResArr_1[i]=corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample, antithetic=True, mean_match=False)
    ResArr_2[i]=corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample, antithetic=False, mean_match=True)
    ResArr_3[i]=corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample, antithetic=True, mean_match=True)
    i=i+1


In [8]:
#Result
print('When antithetic=False, mean_match=False, MC mean and std are ',ResArr_0.mean(),ResArr_0.std())
print('When antithetic=True, mean_match=False, MC mean and std are ',ResArr_1.mean(),ResArr_1.std())
print('When antithetic=False, mean_match=True, MC mean and std are ',ResArr_2.mean(),ResArr_2.std())
print('When antithetic=True, mean_match=True, MC mean and std are ',ResArr_3.mean(),ResArr_3.std())
    

When antithetic=False, mean_match=False, MC mean and std are  0.4399721807668823 0.002167997353448178
When antithetic=True, mean_match=False, MC mean and std are  0.440304419112797 0.0017428190255443091
When antithetic=False, mean_match=True, MC mean and std are  0.4404740070249693 0.0010526786343845154
When antithetic=True, mean_match=True, MC mean and std are  0.4405147616867399 0.0015556257983170962


### 3. Finally, what is the analytic value of the corporate bond? How does it compare to your MC result above?

In [9]:
### Put the analytic expression for the corporate bond price

 price of the bond=NPV of the bond payment=
 $\int_{0}^T R\ \lambda\ e^{-(r_f+\lambda)\ t}dt \ +\ e^{-(r_f+\lambda)\ T}=R\frac{\lambda}{\lambda+r_f} (1-e^{-(r_f+\lambda)T})  + e^{-(r_f+\lambda)T}$

In [10]:
BondPrice=recovery*def_rate/(def_rate+rf_rate)*(1-np.exp(-(def_rate+rf_rate)*mat))+np.exp(-(def_rate+rf_rate)*mat)
(BondPrice,ResArr_0.mean()-BondPrice,ResArr_1.mean()-BondPrice,ResArr_2.mean()-BondPrice,ResArr_3.mean()-BondPrice)

(0.44040907156462505,
 -0.0004368907977427283,
 -0.00010465245182805782,
 6.49354603442509e-05,
 0.00010569012211486539)

Camparison: When using antithetic method, it helps to loose the computation load and slightly reduce the std. For mean_match method, it greatly improve the accuarcy and reduce the std.  However, when combining these two method, the result seems not as good as only using mean_match method.
